### 모델 라이브러리

In [57]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

In [63]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveExtraTrees(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1500)
        , 'max_depth': trial.suggest_int('max_depth', 2, 50)
        , 'min_samples_split': trial.suggest_int('min_samples_split', 2, 5)
        , 'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)
        , 'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy'])
        , 'bootstrap': trial.suggest_categorical('bootstrap', [True, False])
        , 'random_state': 0
    }
    
    model = ExtraTreesClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"].astype(int),
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveExtraTrees(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


[I 2024-02-22 17:25:19,041] A new study created in memory with name: no-name-c0d44ee0-b9cb-4651-ad15-9110854c1441


[I 2024-02-22 17:26:09,990] Trial 0 finished with value: 0.7985480943738658 and parameters: {'n_estimators': 868, 'max_depth': 37, 'min_samples_split': 4, 'min_samples_leaf': 3, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 0 with value: 0.7985480943738658.
[I 2024-02-22 17:27:32,120] Trial 1 finished with value: 0.7922235722964763 and parameters: {'n_estimators': 1450, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 3, 'criterion': 'entropy', 'bootstrap': False}. Best is trial 0 with value: 0.7985480943738658.
[I 2024-02-22 17:27:40,878] Trial 2 finished with value: 0.7857581338244322 and parameters: {'n_estimators': 128, 'max_depth': 42, 'min_samples_split': 5, 'min_samples_leaf': 5, 'criterion': 'gini', 'bootstrap': False}. Best is trial 0 with value: 0.7985480943738658.
[I 2024-02-22 17:28:00,331] Trial 3 finished with value: 0.7838334353949786 and parameters: {'n_estimators': 265, 'max_depth': 33, 'min_samples_split': 2, 'min_samples_leaf': 5, 'criterion'

Best trial: score 0.8374215630347975, 
params {'n_estimators': 100, 'max_depth': 48, 'min_samples_split': 3, 'min_samples_leaf': 1, 'criterion': 'gini', 'bootstrap': False}


### 모델 성능확인

In [64]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [74]:
# ExtraTrees
model = ExtraTreesClassifier(
    n_estimators=100
    , max_depth=48
    , min_samples_split=3
		, min_samples_leaf=1
		, criterion='gini'
		, random_state=0
) 

### 모델 학습

In [75]:
model.fit(x_train, y_train)

ExtraTreesClassifier(max_depth=48, min_samples_split=3, random_state=0)

In [76]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,734,213
False,72,10841


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.97597,0.91067,0.775079,0.837422,0.97508


In [77]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

553

### 제출파일 생성

In [78]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_et_optuna.csv", index=False)

.